## State Pattern

- __Type:__ Behavioral
- __Popularity: ★★★★☆__
- __Complexity: ★★★☆☆__

### Intent:
__State Pattern__ is a behavioral design pattern that allows an object to alter its behavior when its internal state changes. The object will appear to change its class.

### Problem:

You need an object to behave differently depending on its internal state, and the number of states and state transitions could be substantial, making conditional statements complex and error-prone. Specifically:

- Managing object behavior that depends on numerous internal states becomes unwieldy with conditional logic
- State transitions and rules become scattered throughout the code
- Adding new states requires modifying multiple conditional blocks
- The resulting code is difficult to maintain and extend

### Solution:

The State pattern suggests creating state-specific classes for each possible state of an object and extracting all state-specific behaviors into these classes. The context (original object) maintains a reference to the current state object and delegates all state-specific behavior to it.

Key components:

- **Context**: Maintains an instance of a ConcreteState subclass that defines the current state
- **State Interface**: Defines an interface common to all ConcreteState classes
- **Concrete States**: Each implements behavior associated with a particular state

The pattern encapsulates state-specific behavior and makes state transitions explicit. Adding new states becomes as simple as creating new classes that implement the State interface.

### Diagram:

```mermaid
classDiagram
    class Context {
        -state: State
        +request()
        +transitionTo(state)
    }
    class State {
        <<interface>>
        +handle(context)
    }
    class ConcreteStateA {
        +handle(context)
    }
    class ConcreteStateB {
        +handle(context)
    }
    class ConcreteStateC {
        +handle(context)
    }
    
    Context o-- State : contains
    State <|-- ConcreteStateA
    State <|-- ConcreteStateB
    State <|-- ConcreteStateC
```

In [2]:
from abc import ABC, abstractmethod


# The State interface declares methods that all Concrete States should implement
class State(ABC):
    @abstractmethod
    def handle(self, context) -> None:
        pass

    def __str__(self) -> str:
        return self.__class__.__name__

In [3]:
# Concrete States implement various behaviors associated with a state
class DraftState(State):
    def handle(self, context) -> None:
        print("Draft State: Document is being drafted.")
        print("Draft State: Moving to moderation.")
        context.transition_to(ModerationState())


class ModerationState(State):
    def handle(self, context) -> None:
        print("Moderation State: Document is being reviewed.")
        print("Moderation State: Moving to published state.")
        context.transition_to(PublishedState())


class PublishedState(State):
    def handle(self, context) -> None:
        print("Published State: Document is live and viewable by the public.")
        print("Published State: Can move back to draft for revisions.")
        context.transition_to(DraftState())

In [4]:
# The Context defines the interface of interest to clients
class DocumentContext:
    _state = None

    def __init__(self) -> None:
        self.transition_to(DraftState())

    def transition_to(self, state: State) -> None:
        print(f"Context: Transitioning to {state}")
        self._state = state
        self._state.context = self

    def draft(self) -> None:
        self._state.handle(self)

    def moderation(self) -> None:
        self._state.handle(self)

    def publish(self) -> None:
        self._state.handle(self)

In [5]:
# Client code
if __name__ == "__main__":
    # Create context with initial state
    context = DocumentContext()

    # Simulate document workflow
    print("\n[Document Creation]")
    context.draft()

    print("\n[Document Review]")
    context.moderation()

    print("\n[Document Publication]")
    context.publish()

    print("\n[Document Revision]")
    context.draft()

Context: Transitioning to DraftState

[Document Creation]
Draft State: Document is being drafted.
Draft State: Moving to moderation.
Context: Transitioning to ModerationState

[Document Review]
Moderation State: Document is being reviewed.
Moderation State: Moving to published state.
Context: Transitioning to PublishedState

[Document Publication]
Published State: Document is live and viewable by the public.
Published State: Can move back to draft for revisions.
Context: Transitioning to DraftState

[Document Revision]
Draft State: Document is being drafted.
Draft State: Moving to moderation.
Context: Transitioning to ModerationState


### Real-world analogies:

1. **Traffic Light:**
   - Each light state (red, yellow, green) has specific behaviors and rules for transitioning
   - The traffic light system maintains the current state and changes behavior accordingly
   - State transitions follow a predetermined sequence

2. **Vending Machine:**
   - Behavior changes based on internal state (no coin inserted, coin inserted, item selected, etc.)
   - Each state allows only specific actions (can't dispense until payment is made)
   - State transitions occur as a result of user actions and internal processes

### When to use:

- When an object's behavior depends on its state, and it must change behavior at runtime
- When operations have large, multipart conditional statements that depend on the object's state
- When state transitions are complex and follow well-defined rules
- When adding new states should not require changes to existing state classes or the context
- When state-specific behavior tends to change independently from other behaviors

### Benefits:

* **Single Responsibility Principle**: Separates state-specific behavior into different classes
* **Open/Closed Principle**: Introducing new states doesn't affect context or existing states
* **Eliminates Conditional Statements**: Replaces state-dependent if/else or switch statements
* **Improved Organization**: State transitions are explicit and centralized in state classes
* **Thread Safety**: State transitions can be atomic, improving thread safety

### Python-specific implementation notes:

- Python's duck typing makes the formal State interface optional, though it's still good practice
- Python's dynamic nature allows for more flexible state transitions than in static languages
- Several Python libraries provide state machine implementations that can simplify the pattern
- Python's decorator pattern can be combined with State to add behaviors to specific states

### Implementation with State Machine Library

Python offers several state machine libraries that can simplify implementing the State pattern. Two popular libraries are `transitions` and `python-state-machine`, which provide clean APIs for defining states, transitions, and callbacks.

In [6]:
# Install the transitions library if not already installed
# Uncomment and run this cell if needed
# !pip install transitions

In [7]:
from state_machine import (
    Event,
    InvalidStateTransition,
    State,
    acts_as_state_machine,
    after,
    before,
)


@acts_as_state_machine
class Process:
    # Define possible states
    created = State(initial=True)
    waiting = State()
    running = State()
    terminated = State()
    blocked = State()
    swapped_out_waiting = State()
    swapped_out_blocked = State()

    # Define state transitions
    wait = Event(
        from_states=(
            created,
            running,
            blocked,
            swapped_out_waiting,
        ),
        to_state=waiting,
    )
    run = Event(from_states=waiting, to_state=running)
    terminate = Event(from_states=running, to_state=terminated)
    block = Event(
        from_states=(
            running,
            swapped_out_blocked,
        ),
        to_state=blocked,
    )
    swap_wait = Event(
        from_states=waiting,
        to_state=swapped_out_waiting,
    )
    swap_block = Event(
        from_states=blocked,
        to_state=swapped_out_blocked,
    )

    def __init__(self, name):
        self.name = name

    # Define hooks that execute during state transitions
    @after("wait")
    def wait_info(self):
        print(f"{self.name} entered waiting mode")

    @after("run")
    def run_info(self):
        print(f"{self.name} is running")

    @before("terminate")
    def terminate_info(self):
        print(f"{self.name} terminated")

    @after("block")
    def block_info(self):
        print(f"{self.name} is blocked")

    @after("swap_wait")
    def swap_wait_info(self):
        print(f"{self.name} is swapped out and waiting")

    @after("swap_block")
    def swap_block_info(self):
        print(f"{self.name} is swapped out and blocked")


def transition(proc, event, event_name):
    try:
        event()
    except InvalidStateTransition:
        msg = f"Transition of {proc.name} from {proc.current_state} to {event_name} failed"
        print(msg)


def state_info(proc):
    print(f"state of {proc.name}: {proc.current_state}")


In [8]:
def main():
    RUNNING = "running"
    WAITING = "waiting"
    BLOCKED = "blocked"
    TERMINATED = "terminated"

    p1, p2 = Process("process1"), Process("process2")
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p1, p1.wait, WAITING)
    transition(p2, p2.terminate, TERMINATED)
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p1, p1.run, RUNNING)
    transition(p2, p2.wait, WAITING)
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p2, p2.run, RUNNING)
    [state_info(p) for p in (p1, p2)]

    print()
    [transition(p, p.block, BLOCKED) for p in (p1, p2)]
    [state_info(p) for p in (p1, p2)]

    print()
    [transition(p, p.terminate, TERMINATED) for p in (p1, p2)]
    [state_info(p) for p in (p1, p2)]


if __name__ == "__main__":
    main()

state of process1: created
state of process2: created

process1 entered waiting mode
Transition of process2 from created to terminated failed
state of process1: waiting
state of process2: created

process1 is running
process2 entered waiting mode
state of process1: running
state of process2: waiting

process2 is running
state of process1: running
state of process2: running

process1 is blocked
process2 is blocked
state of process1: blocked
state of process2: blocked

Transition of process1 from blocked to terminated failed
Transition of process2 from blocked to terminated failed
state of process1: blocked
state of process2: blocked


### Related patterns:

- **Strategy Pattern**: While State Pattern encapsulates state-dependent behavior, Strategy Pattern encapsulates algorithms. The key difference is that states know about each other, while strategies typically don't.
- **Singleton Pattern**: State objects are often implemented as singletons when they have no instance variables.
- **Flyweight Pattern**: Can be used with State when you have many objects with similar states to reduce memory usage.
- **Command Pattern**: Can be combined with State to implement undo/redo functionality in state transitions.